In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [64]:
ns = np.array([20, 100])
distributions = {
  'Normal': [np.random.normal(loc=0, scale=1, size=n) for n in ns],
  'Student\'s t': [np.random.standard_t(df=10, size=n) for n in ns],
  'Uniform': [np.random.uniform(low=-1, high=1, size=n) for n in ns],
}

In [65]:

Ps = {
  'Normal': {
    5: [3, 4, 6, 4, 3],
    8: [14, 10, 12, 14, 14, 12, 10, 14]
  },
  'Student\'s t': {
    5: [4, 5, 5, 3, 3],
    8: [20, 7, 8, 9, 10, 9, 8, 29]
  },
  'Uniform': {
    5: [0, 6, 7, 7, 0],
    8: [0, 14, 18, 18, 18, 18, 14, 0]
  },
}

In [66]:
bounds = {
  5: [-1.1, -0.367, 0.367, 1.1],
  8: [-1.1, -0.733, 0.367, 0]
}

In [67]:
batches = {
  'Normal': sp.stats.norm,
  'Student\'s t': sp.stats.t,
  'Uniform': sp.stats.uniform
}

In [68]:
def chi2_test(data, bins, dist_name='Normal', alpha=0.05):
  observed_values, _ = np.histogram(data, bins=bins)
  edges = np.histogram_bin_edges(data, bins=bins)
  #print(edges)
  Fxs = batches[dist_name].cdf(edges)
  Fxs[0] = 0
  Fxs[-1] = 1
  #print(Fxs)
  Pis = Fxs - np.roll(Fxs, 1)
  # print(Pis)
  exp_values = [round(len(data) * pi) for pi in Pis[1:]]
  P_vals = Pis[1:]
  diff = abs(np.sum(observed_values) - sum(exp_values))
  obs_sum = np.sum(observed_values)
  exp_sum = sum(exp_values)
  #print(obs_sum, exp_sum)
  #print("diff = ", diff)
  if (sum(exp_values) != len(data)):
    for i in range(0, len(exp_values)):
      if (observed_values[i] != exp_values[i]):
        if (observed_values[i] < exp_values[i] and obs_sum < exp_sum):
          observed_values[i] = observed_values[i] + diff
          break
        elif (observed_values[i] > exp_values[i] and obs_sum > exp_sum):
          exp_values[i] = exp_values[i] + diff
          break
  #print(observed_values)
  #print(exp_values)
  # print(np.sum(observed_values), sum(exp_values))
  # expected_values = [len(data)/bins] * bins
  expected_values = Ps[dist_name][bins]
  chi2, p_val = sp.stats.chisquare(f_obs=observed_values, f_exp=exp_values)
  
  chi2_c = 0
  for i in range(len(P_vals)):
    chi2_c = chi2_c + ((observed_values[i] - len(data) * P_vals[i]) ** 2) / (len(data) * P_vals[i])
  chi2_critical = sp.stats.chi2.ppf(q = 1 - alpha, df = bins - 1)

  return chi2_c, p_val, chi2_critical

In [69]:
alpha = 0.05
bins = 10

for dist_name, datasets in distributions.items():
  for i, data in enumerate(datasets):
    if (len(data) == 20):
      bins = 5
    else:
      bins = 8
    chi2, p_val, chi2_critical = chi2_test(data, bins)
    print(f"\nDistribution: {dist_name}, n={ns[i]}")
    print(f"Chi-squared: {chi2}, chi2-critical: {chi2_critical}")
    if chi2 < chi2_critical:
      print(f"Do not reject null hypothesis, data follows the expected distribution")
    else:
      print(f"Reject null hypothesis, data does not follow the expected distribution")


Distribution: Normal, n=20
Chi-squared: 3.792180331468197, chi2-critical: 9.487729036781154
Do not reject null hypothesis, data follows the expected distribution

Distribution: Normal, n=100
Chi-squared: 9.818978679343038, chi2-critical: 14.067140449340167
Do not reject null hypothesis, data follows the expected distribution

Distribution: Student's t, n=20
Chi-squared: 8.977658365681485, chi2-critical: 9.487729036781154
Do not reject null hypothesis, data follows the expected distribution

Distribution: Student's t, n=100
Chi-squared: 9.542928288352147, chi2-critical: 14.067140449340167
Do not reject null hypothesis, data follows the expected distribution

Distribution: Uniform, n=20
Chi-squared: 4.760333627552156, chi2-critical: 9.487729036781154
Do not reject null hypothesis, data follows the expected distribution

Distribution: Uniform, n=100
Chi-squared: 14.89576874374985, chi2-critical: 14.067140449340167
Reject null hypothesis, data does not follow the expected distribution
